# Úkol č. 2 - předzpracování dat a binární klasifikace (do 10. listopadu)

  * Cílem thoto úkolu je vyzkoušet si naučit prediktivní model pro binární klasifikaci.
  * Budete se muset vypořádat s příznaky, které jsou různých typů a které bude třeba nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu (preferujeme ale použití cross-validation).
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací/vaidační množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. **První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [133]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

import warnings

In [2]:
data = pd.read_csv('data.csv')
df = data.drop(columns=["name"])
df["sex"].replace(['male','female'],[True,False],inplace=True)
# první písmena kabiny převede na čísla, kde žádná hodnota je 0 a poté postupně G - A se převede na 1 - 7
cabin_level = {
    "1": "G",
    "2": "F",
    "3": "E",
    "4": "D",
    "5": "C",
    "6": "B", 
    "7": "A"
}
df['cabin'] = np.where(df['cabin'].isnull(), '0', df['cabin'])
for index, value  in cabin_level.items():
   df['cabin'] = np.where(df['cabin'].str.contains(value), index, df['cabin'])
df['cabin'] = pd.to_numeric(df['cabin'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
ID           1000 non-null int64
survived     1000 non-null int64
pclass       1000 non-null int64
sex          1000 non-null bool
age          799 non-null float64
sibsp        1000 non-null int64
parch        1000 non-null int64
ticket       1000 non-null object
fare         999 non-null float64
cabin        1000 non-null int64
embarked     998 non-null object
home.dest    565 non-null object
dtypes: bool(1), float64(2), int64(6), object(3)
memory usage: 87.0+ KB


In [4]:
# řádky s nekompletní hodnotou ve sloupcích fare a embarked vyhodím, protože 3 z 1000 je poměrně málo
df2 = df.loc[df.fare.notnull()]
df2 = df2.loc[df.embarked.notnull()]
df2.info()
home_dest_unique = df2.loc[df["home.dest"].notnull()].nunique()
home_dest_unique

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 999
Data columns (total 12 columns):
ID           997 non-null int64
survived     997 non-null int64
pclass       997 non-null int64
sex          997 non-null bool
age          796 non-null float64
sibsp        997 non-null int64
parch        997 non-null int64
ticket       997 non-null object
fare         997 non-null float64
cabin        997 non-null int64
embarked     997 non-null object
home.dest    564 non-null object
dtypes: bool(1), float64(2), int64(6), object(3)
memory usage: 94.4+ KB


ID           564
survived       2
pclass         3
sex            2
age           80
sibsp          6
parch          7
ticket       411
fare         192
cabin          8
embarked       3
home.dest    318
dtype: int64

In [84]:
# změna 'home.dest' na sloupce fromUSA a notFromUSA, odhad podle regext '.*, ..$' napr 'Ireland New York, NY'
data["home.dest"]
df3 = df2.replace(to_replace ='.*, ..$', value = 'USA', regex = True) 
home_dest_column = df3["home.dest"]
home_dest_column = home_dest_column.replace(['.+[^U][^S][^A]$', '.*'], [1, 0], regex = True)
home_dest_column = home_dest_column.replace(np.nan, 0)
home_dest_column
df3["notFromUSA"] = home_dest_column
df3["home.dest"] = df3["home.dest"].replace(['USA$', '.*'], [1, 0], regex = True)
df3["home.dest"] = df3["home.dest"].replace(np.nan, 0)
df3.rename(columns = {'home.dest': 'fromUSA'}, inplace=True)
df3['notFromUSA'] = pd.to_numeric(df3['notFromUSA'], downcast='integer')
df3['fromUSA'] = pd.to_numeric(df3['fromUSA'], downcast='integer')
df3["age"] = df3["age"].replace(np.nan, df3['age'].median())
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 999
Data columns (total 13 columns):
ID            997 non-null int64
survived      997 non-null int64
pclass        997 non-null int64
sex           997 non-null bool
age           997 non-null float64
sibsp         997 non-null int64
parch         997 non-null int64
ticket        997 non-null object
fare          997 non-null float64
cabin         997 non-null int64
embarked      997 non-null object
fromUSA       997 non-null int8
notFromUSA    997 non-null int8
dtypes: bool(1), float64(2), int64(6), int8(2), object(2)
memory usage: 88.6+ KB


In [96]:
df4 = df3.drop(columns = 'ticket')
df4.rename(columns = {'embarked': 'embarked S'}, inplace=True)
df4['embarked C'] = df4['embarked S'].replace(['C', '.'], [1, 0], regex = True)
df4['embarked Q'] = df4['embarked S'].replace(['Q', '.'], [1, 0], regex = True)
df4['embarked S'] = df4['embarked S'].replace(['S', '.'], [1, 0], regex = True)
#df4['Y survived'] = df4['survived']
#df4 = df4.drop(columns = 'survived')
df4

,ID,survived,pclass,sex,age,sibsp,parch,fare,cabin,embarked S,fromUSA,notFromUSA,embarked C,embarked Q
0,0,1,3,False,22.0,0,0,7.7500,0,1,0,0,0,0
1,1,0,3,True,28.0,0,0,8.4583,0,0,0,0,0,1
2,2,1,1,False,19.0,1,0,91.0792,6,0,1,0,1,0
3,3,0,3,True,25.0,0,0,7.2250,0,0,0,0,1,0
4,4,0,3,True,28.0,0,0,7.7500,0,0,1,0,0,1
5,5,0,1,True,27.0,0,2,211.5000,5,0,1,0,1,0
6,6,1,1,False,24.0,3,2,263.0000,5,1,1,0,0,0
7,7,1,3,False,28.0,1,0,16.1000,0,1,1,0,0,0
8,8,0,3,True,61.0,0,0,6.2375,0,1,0,0,0,0
9,9,0,3,True,8.0,4,1,29.1250,0,0,0,0,0,1


In [121]:
def split_data2(Xdata, ydata, ratio=0.25, rd_seed=5656):
    Xtrain, Xval, ytrain, yval = train_test_split(Xdata, ydata, test_size=0.25, random_state=rd_seed) 
    return Xtrain, Xval, ytrain, yval

In [161]:
def split_data(Xdata, ydata, ratio=0.25, rd_seed=5656):
    Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.25, random_state=rd_seed) 
    Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed) 
    return Xtrain, Xtest, Xval, ytrain, ytest, yval

In [163]:
#Xtrain, Xval, ytrain, yval = split_data2(df4.drop(columns=['survived']), df4.survived)
Xtrain, Xtest, Xval, ytrain, ytest, yval = split_data(df4.drop(columns=['survived']), df4.survived)

In [177]:
param_grid = {
    'n_estimators': range(1,40,1),
    'max_depth': range(1,8)
}
param_comb = ParameterGrid(param_grid)
val_acc = []
for params in param_comb:
    params['random_state'] = 3654
    dt = RandomForestClassifier(**params)
    dt.fit(Xtrain, ytrain)
    val_acc.append(metrics.accuracy_score(yval, dt.predict(Xval)))
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(max(val_acc))
dt = RandomForestClassifier(**best_params, random_state = 3654)
dt

{'n_estimators': 23, 'max_depth': 7}
0.8074866310160428


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=23,
                       n_jobs=None, oob_score=False, random_state=3654,
                       verbose=0, warm_start=False)

In [178]:
dt = RandomForestClassifier(**best_params)
dt.fit(Xtrain, ytrain)
metrics.accuracy_score(ytest, dt.predict(Xtest))

0.792